In [12]:
# -*- coding: utf-8 -*-
"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input='train_data/labels.csv'  --output_path='data/train.record'
  # Create test data:
  python generate_tfrecord.py --csv_input=test_data/labels.csv  --output_path=data/test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import sys
import pandas as pd
import tensorflow as tf

from PIL import Image

# sys.path.append("..")

from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

# TO-DO replace this with label map
def class_text_to_int(row_label):
    print(row_label, "\n")
    if row_label == 'hanwha':
        print("baaam", "\n")
        return 1
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    print('++++++++++++', group, path)
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter('data/test.record')
    print('-----------------------writer----------------------------')
    path = os.path.join(os.getcwd(), 'images/test_data/')
    examples = pd.read_csv('images/test_data/labels.csv')
    grouped = split(examples, 'filename')
    for group in grouped:
        print('-----------------------group----------------------------')
        tf_example = create_tf_example(group, path)
        print('-----------------------create tf_example----------------------------')
        writer.write(tf_example.SerializeToString())
        print('-----------------------writer.write----------------------------')

    writer.close()
    print('-----------------------writer.close----------------------------')
    
    output_path = os.path.join(os.getcwd(), 'data/test.record')
    print('Successfully created the TFRecords: {}'.format(output_path))

In [13]:
tf.app.run()

-----------------------writer----------------------------
-----------------------group----------------------------
++++++++++++ data(filename='techwin.jpg', object=      filename  width  height   class  xmin  ymin  xmax  ymax
0  techwin.jpg    309     138  hanwha     6    17   102   108) C:\models\research\object_detection\images/test_data/
hanwha 

baaam 

-----------------------create tf_example----------------------------
-----------------------writer.write----------------------------
-----------------------writer.close----------------------------
Successfully created the TFRecords: C:\models\research\object_detection\data/test.record


SystemExit: 

c:\programdata\anaconda3\envs\hsncml01\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
